<a href="https://colab.research.google.com/github/YellaCRE/SportRecog/blob/main/%EB%AC%B8%ED%99%94_sportEDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U pandas-profiling

In [ ]:
!pip install MarkupSafe==2.0.1

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#데이터 불러오기

In [ ]:
import os
import pandas as pd
import pandas_profiling
pd.set_option('mode.chained_assignment',  None) #수정오류 무시
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
path = "/content/drive/MyDrive/2022 휴학/문화관광공모전/data/"
#csv 형식으로 된 데이터 파일을 읽어옵니다. 
data = pd.read_csv(path + "person_data.csv")
data.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [ ]:
df_covid = data[data['year'].isin([2021,2020])]
df_norm = data[data['year'].isin([2019,2018])]

In [ ]:
col_list = data.columns
def exp(data):
  df_exp = data[col_list[col_list.str.contains('Q1_')==True]]
  data.drop(col_list[col_list.str.contains('Q1_')==True], axis = 1, inplace = True)
  return df_exp

df_covid_exp = exp(df_covid)
df_norm_exp = exp(df_norm)

In [ ]:
def Counth(data):
  h1 = data.groupby(['Q4_1']).size().sort_values(ascending=False)
  h2 = data.groupby(['Q4_2']).size().sort_values(ascending=False)
  h3 = data.groupby(['Q4_3']).size().sort_values(ascending=False)
  total = h1+h2+h3
  top5 = ['TV시청(IPTV 포함)','산책 및 걷기','친구만남/이성교제/미팅/소개팅','쇼핑/외식','모바일 컨텐츠, 동영상, VOD 시청','인터넷 검색/1인 미디어 제작/SNS','잡담/통화하기/문자보내기(모바일 메신저 메시지 사용 포함)','음주']
  total.drop(top5,inplace=True)
  return total.sort_values(ascending=False).head(10)

#EDA

In [ ]:
df_covid.head()

,ID,year,DM1,DM2,DM11,Q45,Q47_N,DM5,Q3,Q4_1,...,Q11_1_D,Q11_2_D,Q11_3_C,Q11_3_D,Q12_1_C,Q12_2_C,Q12_1_D,Q12_2_D,Q12_3_C,Q12_3_D
20558,20201,2020,F,60,충북,하였다,40.0,기혼,즐거움,취미생활중,...,헬스(보디빌딩)/에어로빅,6.0,"스포츠 경기 간접관람- TV, DMB를 통한관람(축구, 야구, 농구, 배구 등)",요가/필라테스/태보,기타,NaN,기타,NaN,"스포츠 경기 간접관람- TV, DMB를 통한관람(축구, 야구, 농구, 배구 등)",요가/필라테스/태보
20559,20202,2020,F,60,충북,하였다,40.0,기혼,휴식,사교활동,...,사이클링/산악자전거,6.0,기타,사이클링/산악자전거,기타,NaN,기타,NaN,기타,"윈드서핑, 수상스키, 스킨스쿠버다이빙, 래프팅, 요트"
20560,20203,2020,F,50,충북,하였다,45.0,기혼,휴식,취미생활중,...,기타,NaN,"스포츠 경기 간접관람- TV, DMB를 통한관람(축구, 야구, 농구, 배구 등)",요가/필라테스/태보,기타,NaN,기타,NaN,"스포츠 경기 간접관람- TV, DMB를 통한관람(축구, 야구, 농구, 배구 등)",헬스(보디빌딩)/에어로빅
20561,20204,2020,M,70,충북,하지 않았다,28.0,기혼,휴식,취미생활중,...,기타,NaN,기타,기타,기타,NaN,기타,NaN,기타,기타
20562,20205,2020,M,70,충북,하였다,40.0,기혼,자기계발,취미생활중,...,기타,NaN,기타,기타,기타,NaN,기타,NaN,기타,기타


In [ ]:
df_covid.Q4_1.unique()

array(['취미생활중', '사교활동', '취미생활여', '예술체험', '여행', '취미생활남', '골프', '예술관람',
       '스포츠 경기 직접관람- 경기장방문관람(축구, 야구,농구, 배구 등)', '요가/필라테스/태보', '수영',
       '배드민턴/줄넘기/맨손, 스트레칭 체조/훌라후프', '헬스(보디빌딩)/에어로빅', '등산', '기타',
       '스포츠 경기 간접관람- TV, DMB를 통한관람(축구, 야구, 농구, 배구 등)',
       '농구, 배구, 야구, 축구, 족구', '격투 스포츠(태권도, 유도, 합기도, 검도, 권투 등)', '테니스, 스쿼시',
       '볼링, 탁구', '육상/조깅/속보', '사이클링/산악자전거', '당구/포켓볼',
       '온라인게임 경기관람(e-스포츠 경기 포함)', '윈드서핑, 수상스키, 스킨스쿠버다이빙, 래프팅, 요트',
       '댄스스포츠(탱고, 왈츠, 자이브, 맘보,폴카, 차차차등)', '승마, 암벽등반, 철인삼종경기, 서바이벌',
       '스노보드, 스키 등', '격투 스포츠 경기관람(태권도, 유도, 합기도, 검도,권투 등)', '인라인스케이트',
       '아이스스케이트, 아이스하키 등'], dtype=object)

#체력에 대한 인식

소득과 건강은 큰 의미가 없는듯

In [ ]:
path = "/content/drive/MyDrive/2022 휴학/문화관광공모전/data/"
#csv 형식으로 된 데이터 파일을 읽어옵니다. 
muscle = pd.read_csv(path + "체력인식_긍정합계.csv")
muscle.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [ ]:
muscle[muscle['category']=='10대']

,category,2021,2020,2019,2018,gender
6,10대,80.4,81.6,83.6,83.9,all
13,10대,86.3,85.2,88.1,88.4,남성
20,10대,74.1,77.7,78.7,79.1,여성


In [ ]:
muscle[muscle['category']=='20대']

,category,2021,2020,2019,2018,gender
7,20대,73.0,80.2,83.9,81.0,all
14,20대,83.3,86.2,89.0,84.3,남성
21,20대,61.5,73.5,78.2,77.3,여성


In [ ]:
muscle[muscle['category']=='30대']

,category,2021,2020,2019,2018,gender
8,30대,70.9,73.2,72.2,75.5,all
15,30대,78.8,78.1,77.2,83.3,남성
22,30대,62.1,67.8,66.9,67.0,여성


In [ ]:
muscle[muscle['category']=='40대']

,category,2021,2020,2019,2018,gender
9,40대,62.7,69.1,65.9,66.3,all
16,40대,70.6,73.7,70.1,69.1,남성
23,40대,54.5,64.3,61.4,63.5,여성


In [ ]:
muscle[muscle['category']=='50대']

,category,2021,2020,2019,2018,gender
10,50대,50.7,59.2,57.0,60.8,all
17,50대,58.7,63.1,63.8,64.0,남성
24,50대,42.6,55.3,50.1,57.7,여성


In [ ]:
muscle[muscle['category']=='60대']

,category,2021,2020,2019,2018,gender
11,60대,36.0,48.6,40.6,46.1,all
18,60대,41.1,54.8,47.7,50.3,남성
25,60대,31.2,42.7,33.8,42.1,여성


#운동 여가활동

In [ ]:
#스포츠 리스트 생성
Q1_name = pd.read_csv(path + 'Q1_colname.csv')
sport = Q1_name[14:37].컬럼명.to_list()
sport.append(Q1_name[54:55].컬럼명.iloc[0])
sport.append(Q1_name[70:71].컬럼명.iloc[0])

In [ ]:
def Countsport(data):
  h1 = data.groupby(['Q4_1']).size().sort_values(ascending=False)
  h2 = data.groupby(['Q4_2']).size().sort_values(ascending=False)
  h3 = data.groupby(['Q4_3']).size().sort_values(ascending=False)
  total = h1+h2+h3
  return pd.DataFrame(total, columns=['count']).loc[total.index.isin(sport)].sort_values('count',ascending=False)

#df_covid

In [ ]:
df = df_covid
df_exp = df_covid_exp

In [ ]:
#df.profile_report()

In [ ]:
df.head()

,ID,year,DM1,DM2,DM11,Q45,Q47_N,DM5,Q3,Q4_1,...,Q11_1_D,Q11_2_D,Q11_3_C,Q11_3_D,Q12_1_C,Q12_2_C,Q12_1_D,Q12_2_D,Q12_3_C,Q12_3_D
20558,20201,2020,F,60,충북,하였다,40.0,기혼,즐거움,취미생활중,...,헬스(보디빌딩)/에어로빅,6.0,"스포츠 경기 간접관람- TV, DMB를 통한관람(축구, 야구, 농구, 배구 등)",요가/필라테스/태보,기타,NaN,기타,NaN,"스포츠 경기 간접관람- TV, DMB를 통한관람(축구, 야구, 농구, 배구 등)",요가/필라테스/태보
20559,20202,2020,F,60,충북,하였다,40.0,기혼,휴식,사교활동,...,사이클링/산악자전거,6.0,기타,사이클링/산악자전거,기타,NaN,기타,NaN,기타,"윈드서핑, 수상스키, 스킨스쿠버다이빙, 래프팅, 요트"
20560,20203,2020,F,50,충북,하였다,45.0,기혼,휴식,취미생활중,...,기타,NaN,"스포츠 경기 간접관람- TV, DMB를 통한관람(축구, 야구, 농구, 배구 등)",요가/필라테스/태보,기타,NaN,기타,NaN,"스포츠 경기 간접관람- TV, DMB를 통한관람(축구, 야구, 농구, 배구 등)",헬스(보디빌딩)/에어로빅
20561,20204,2020,M,70,충북,하지 않았다,28.0,기혼,휴식,취미생활중,...,기타,NaN,기타,기타,기타,NaN,기타,NaN,기타,기타
20562,20205,2020,M,70,충북,하였다,40.0,기혼,자기계발,취미생활중,...,기타,NaN,기타,기타,기타,NaN,기타,NaN,기타,기타


EDA

In [ ]:
Countsport(df)

,count
등산,1740
헬스(보디빌딩)/에어로빅,1297
"스포츠 경기 간접관람- TV, DMB를 통한관람(축구, 야구, 농구, 배구 등)",890
"농구, 배구, 야구, 축구, 족구",675
요가/필라테스/태보,619
"배드민턴/줄넘기/맨손, 스트레칭 체조/훌라후프",604
골프,557
"볼링, 탁구",401
당구/포켓볼,401
사이클링/산악자전거,386


In [ ]:
#거리두기 이후 스포츠여가활동
Countsport(df).sum()

count    8909
dtype: int64

성별에 따른 여가활동

In [ ]:
Countsport(df[df['DM1']=='M'])

,count
등산,1178
헬스(보디빌딩)/에어로빅,822
"스포츠 경기 간접관람- TV, DMB를 통한관람(축구, 야구, 농구, 배구 등)",822
"농구, 배구, 야구, 축구, 족구",655
골프,418
당구/포켓볼,379
사이클링/산악자전거,300
"볼링, 탁구",297
"배드민턴/줄넘기/맨손, 스트레칭 체조/훌라후프",286
"스포츠 경기 직접관람- 경기장방문관람(축구, 야구,농구, 배구 등)",178


In [ ]:
Countsport(df[df['DM1']=='F'])

,count
요가/필라테스/태보,596.0
등산,562.0
헬스(보디빌딩)/에어로빅,475.0
"배드민턴/줄넘기/맨손, 스트레칭 체조/훌라후프",318.0
수영,241.0
골프,139.0
"볼링, 탁구",104.0
사이클링/산악자전거,86.0
육상/조깅/속보,75.0
"스포츠 경기 간접관람- TV, DMB를 통한관람(축구, 야구, 농구, 배구 등)",68.0


여가활동의 목적

In [ ]:
df.groupby(['Q3']).size().sort_values(ascending=False)

Q3
즐거움     10793
휴식       5457
자기계발     2844
대인관계     1031
기타         12
dtype: int64

나이에 따른 여가생활

In [ ]:
df.groupby(['DM2']).size().sort_values(ascending=False)

DM2
50    3758
40    3670
30    3136
20    3048
60    2854
70    2464
10    1207
dtype: int64

Z세대

In [ ]:
Countsport(df[df['DM2'].isin([10])])

,count
"농구, 배구, 야구, 축구, 족구",156.0
헬스(보디빌딩)/에어로빅,62.0
"스포츠 경기 간접관람- TV, DMB를 통한관람(축구, 야구, 농구, 배구 등)",47.0
"배드민턴/줄넘기/맨손, 스트레칭 체조/훌라후프",44.0
"격투 스포츠(태권도, 유도, 합기도, 검도, 권투 등)",37.0
"볼링, 탁구",35.0
"스포츠 경기 직접관람- 경기장방문관람(축구, 야구,농구, 배구 등)",20.0
사이클링/산악자전거,20.0
당구/포켓볼,16.0
온라인게임 경기관람(e-스포츠 경기 포함),11.0


MZ세대

In [ ]:
Countsport(df[df['DM2'].isin([20,30])])

,count
헬스(보디빌딩)/에어로빅,669.0
"농구, 배구, 야구, 축구, 족구",317.0
"스포츠 경기 간접관람- TV, DMB를 통한관람(축구, 야구, 농구, 배구 등)",267.0
요가/필라테스/태보,262.0
"볼링, 탁구",186.0
등산,171.0
당구/포켓볼,161.0
"배드민턴/줄넘기/맨손, 스트레칭 체조/훌라후프",145.0
골프,106.0
수영,103.0


Y세대

In [ ]:
Countsport(df[df['DM2'].isin([30,40])])

,count
헬스(보디빌딩)/에어로빅,500.0
등산,432.0
"스포츠 경기 간접관람- TV, DMB를 통한관람(축구, 야구, 농구, 배구 등)",356.0
요가/필라테스/태보,274.0
골프,242.0
"농구, 배구, 야구, 축구, 족구",219.0
"배드민턴/줄넘기/맨손, 스트레칭 체조/훌라후프",193.0
당구/포켓볼,173.0
"볼링, 탁구",146.0
수영,141.0


X세대

In [ ]:
Countsport(df[df['DM2'].isin([50,60])])

,count
등산,1071.0
"스포츠 경기 간접관람- TV, DMB를 통한관람(축구, 야구, 농구, 배구 등)",318.0
헬스(보디빌딩)/에어로빅,309.0
골프,267.0
"배드민턴/줄넘기/맨손, 스트레칭 체조/훌라후프",210.0
요가/필라테스/태보,162.0
사이클링/산악자전거,158.0
수영,144.0
당구/포켓볼,109.0
"농구, 배구, 야구, 축구, 족구",93.0


베이비붐세대

In [ ]:
Countsport(df[df['DM2'].isin([60,70])])

,count
등산,641.0
"스포츠 경기 간접관람- TV, DMB를 통한관람(축구, 야구, 농구, 배구 등)",181.0
"배드민턴/줄넘기/맨손, 스트레칭 체조/훌라후프",168.0
헬스(보디빌딩)/에어로빅,146.0
수영,112.0
요가/필라테스/태보,107.0
사이클링/산악자전거,83.0
골프,74.0
육상/조깅/속보,54.0
당구/포켓볼,45.0


#df_norm

In [ ]:
df = df_norm
df_exp = df_norm_exp

In [ ]:
#df.profile_report()

##EDA

In [ ]:
Countsport(df)

,count
등산,1891
헬스(보디빌딩)/에어로빅,1372
"스포츠 경기 간접관람- TV, DMB를 통한관람(축구, 야구, 농구, 배구 등)",1023
"스포츠 경기 직접관람- 경기장방문관람(축구, 야구,농구, 배구 등)",705
"농구, 배구, 야구, 축구, 족구",655
요가/필라테스/태보,654
수영,517
"배드민턴/줄넘기/맨손, 스트레칭 체조/훌라후프",515
골프,503
"볼링, 탁구",434


In [ ]:
#거리두기 이전 스포츠여가활동
Countsport(df).sum()

count    9822
dtype: int64

성별에 따른 여가활동

In [ ]:
Countsport(df[df['DM1']=='M'])

,count
등산,1315.0
"스포츠 경기 간접관람- TV, DMB를 통한관람(축구, 야구, 농구, 배구 등)",943.0
헬스(보디빌딩)/에어로빅,808.0
"농구, 배구, 야구, 축구, 족구",640.0
"스포츠 경기 직접관람- 경기장방문관람(축구, 야구,농구, 배구 등)",556.0
당구/포켓볼,386.0
골프,385.0
"볼링, 탁구",316.0
사이클링/산악자전거,273.0
"배드민턴/줄넘기/맨손, 스트레칭 체조/훌라후프",265.0


In [ ]:
Countsport(df[df['DM1']=='F'])

,count
요가/필라테스/태보,619.0
등산,576.0
헬스(보디빌딩)/에어로빅,564.0
수영,352.0
"배드민턴/줄넘기/맨손, 스트레칭 체조/훌라후프",250.0
"스포츠 경기 직접관람- 경기장방문관람(축구, 야구,농구, 배구 등)",149.0
"볼링, 탁구",118.0
골프,118.0
"댄스스포츠(탱고, 왈츠, 자이브, 맘보,폴카, 차차차등)",111.0
"스포츠 경기 간접관람- TV, DMB를 통한관람(축구, 야구, 농구, 배구 등)",80.0


여가활동의 목적

In [ ]:
df.groupby(['Q3']).size().sort_values(ascending=False)

Q3
즐거움     10094
휴식       5062
자기계발     3650
대인관계     1748
기타          4
dtype: int64

In [ ]:
Countsport(df[df['Q3']=='즐거움'])

,count
등산,837
헬스(보디빌딩)/에어로빅,627
"스포츠 경기 간접관람- TV, DMB를 통한관람(축구, 야구, 농구, 배구 등)",552
"농구, 배구, 야구, 축구, 족구",395
"스포츠 경기 직접관람- 경기장방문관람(축구, 야구,농구, 배구 등)",365
요가/필라테스/태보,286
당구/포켓볼,251
골프,248
"배드민턴/줄넘기/맨손, 스트레칭 체조/훌라후프",242
"볼링, 탁구",239


In [ ]:
Countsport(df[df['Q3']=='휴식'])

,count
등산,407.0
헬스(보디빌딩)/에어로빅,253.0
"스포츠 경기 간접관람- TV, DMB를 통한관람(축구, 야구, 농구, 배구 등)",250.0
요가/필라테스/태보,145.0
"스포츠 경기 직접관람- 경기장방문관람(축구, 야구,농구, 배구 등)",116.0
"배드민턴/줄넘기/맨손, 스트레칭 체조/훌라후프",116.0
수영,95.0
"농구, 배구, 야구, 축구, 족구",85.0
골프,76.0
"볼링, 탁구",76.0


나이에 따른 여가생활

In [ ]:
df.groupby(['DM2']).size().sort_values(ascending=False)

DM2
50    3884
40    3883
30    3203
20    2926
60    2746
70    2559
10    1357
dtype: int64

Z세대

In [ ]:
Countsport(df[df['DM2'].isin([10])])

,count
"농구, 배구, 야구, 축구, 족구",191.0
"스포츠 경기 직접관람- 경기장방문관람(축구, 야구,농구, 배구 등)",74.0
"스포츠 경기 간접관람- TV, DMB를 통한관람(축구, 야구, 농구, 배구 등)",56.0
"배드민턴/줄넘기/맨손, 스트레칭 체조/훌라후프",36.0
"볼링, 탁구",35.0
헬스(보디빌딩)/에어로빅,32.0
"격투 스포츠(태권도, 유도, 합기도, 검도, 권투 등)",28.0
온라인게임 경기관람(e-스포츠 경기 포함),16.0
당구/포켓볼,12.0
사이클링/산악자전거,11.0


MZ세대

In [ ]:
Countsport(df[df['DM2'].isin([20,30])])

,count
헬스(보디빌딩)/에어로빅,559.0
"스포츠 경기 직접관람- 경기장방문관람(축구, 야구,농구, 배구 등)",334.0
요가/필라테스/태보,291.0
"스포츠 경기 간접관람- TV, DMB를 통한관람(축구, 야구, 농구, 배구 등)",288.0
"농구, 배구, 야구, 축구, 족구",275.0
"볼링, 탁구",178.0
당구/포켓볼,173.0
수영,155.0
등산,139.0
"배드민턴/줄넘기/맨손, 스트레칭 체조/훌라후프",106.0


Y세대

In [ ]:
Countsport(df[df['DM2'].isin([30,40])])

,count
헬스(보디빌딩)/에어로빅,696.0
등산,486.0
"스포츠 경기 간접관람- TV, DMB를 통한관람(축구, 야구, 농구, 배구 등)",449.0
요가/필라테스/태보,339.0
"스포츠 경기 직접관람- 경기장방문관람(축구, 야구,농구, 배구 등)",330.0
골프,238.0
"농구, 배구, 야구, 축구, 족구",214.0
수영,206.0
당구/포켓볼,187.0
"배드민턴/줄넘기/맨손, 스트레칭 체조/훌라후프",168.0


X세대

In [ ]:
Countsport(df[df['DM2'].isin([50,60])])

,count
등산,1203.0
헬스(보디빌딩)/에어로빅,363.0
"스포츠 경기 간접관람- TV, DMB를 통한관람(축구, 야구, 농구, 배구 등)",341.0
골프,239.0
"배드민턴/줄넘기/맨손, 스트레칭 체조/훌라후프",191.0
수영,186.0
요가/필라테스/태보,164.0
"스포츠 경기 직접관람- 경기장방문관람(축구, 야구,농구, 배구 등)",127.0
"볼링, 탁구",117.0
당구/포켓볼,112.0


베이비붐세대

In [ ]:
Countsport(df[df['DM2'].isin([60,70])])

,count
등산,663.0
"스포츠 경기 간접관람- TV, DMB를 통한관람(축구, 야구, 농구, 배구 등)",185.0
헬스(보디빌딩)/에어로빅,165.0
"배드민턴/줄넘기/맨손, 스트레칭 체조/훌라후프",155.0
수영,131.0
요가/필라테스/태보,103.0
사이클링/산악자전거,71.0
골프,69.0
육상/조깅/속보,56.0
"볼링, 탁구",55.0
